**Notebook to cleanup ECR data from ECR List on Sharepoint**

In [1]:
from pyspark.sql.functions import*
from pyspark.sql.types import StructType,StructField,StringType,ArrayType,IntegerType

df = spark.sql("SELECT * FROM sharepoint_ecr_list.ECR_List_Raw")
df = df.sort(df['ECR_#'].desc())
#display(df)

StatementMeta(, cfa91a5a-fb11-439d-904a-a633ee1fa92e, 3, Finished, Available, Finished)

JSON schema for project and subproject

In [2]:
products_schema = ArrayType(
    StructType()
    .add(StructField('__id',StringType(),True))\
    .add(StructField('Project',StringType(),True))\
    .add(StructField('Subproject',StringType(),True))
)

StatementMeta(, cfa91a5a-fb11-439d-904a-a633ee1fa92e, 4, Finished, Available, Finished)

In [3]:
added_schema = ArrayType(
    StructType()
    .add(StructField('__id',StringType(),True))\
    .add(StructField('Product',StringType(),True))\
    .add(StructField('Drawing',StringType(),True))\
    .add(StructField('Description',StringType(),True))\
    .add(StructField('Qty',IntegerType(),True))\
    .add(StructField('Prod',StringType(),True))
)

StatementMeta(, cfa91a5a-fb11-439d-904a-a633ee1fa92e, 5, Finished, Available, Finished)

Selecting the required columns for data analysis

In [4]:
newdf = df.select(
    col('Title').alias('ECR#'),\
    col('Urgency_Code_Mech'),\
    explode(from_json(col('Products_Mech'),products_schema)).alias('Products_Mech'),\
    col('Products_Added_Mech').alias('Products_Added'),\
    col('Drawing_Mech'),\
    col('Stage_Mech'),\
    col('Created'),\
    col('Modified'),\
    col('Project_Leader_Sign_Mech'),\
    col('ECR_Completed_Sign_Mech'),\
    col('Review_Project_Leader_Sign_Mech'),\
    col('Docu_Control_Leader_Sign_Mech'),\
    col('Docu_Control_Admin_Sign_Mech'),\
    col('NCR#'),\
    col('Total#ofFolders'),\
    col('ReasonforECR0').alias('Reason_For_ECR')
)

#display(newdf)

StatementMeta(, cfa91a5a-fb11-439d-904a-a633ee1fa92e, 6, Finished, Available, Finished)

Splitting the Products_Mech to projects and subprojects columns

In [5]:
#newdf = newdf.withColumn('ID', col('Products_Mech.__id'))
newdf = newdf.withColumn('Project', col('Products_Mech.Project'))
newdf = newdf.withColumn('Subproject', col('Products_Mech.Subproject'))
folderdf = newdf.select(
    col('ECR#'),\
    col('Urgency_Code_Mech'),\
    col('Stage_Mech'),\
    col('Created'),\
    col('Modified'),\
    col('Total#ofFolders'),\
    col('Project'),\
    col('Subproject')
)
#display(folderdf)

StatementMeta(, cfa91a5a-fb11-439d-904a-a633ee1fa92e, 7, Finished, Available, Finished)

Formatting the Project and Subproject columns to identify all the projects and subprojects involved in the ECR

In [6]:
newdf = newdf.withColumn('Project',trim(newdf.Project))
newdf = newdf.withColumn('Project',split('Project','[/,&;]'))
newdf = newdf.withColumn('Project',explode('Project'))
newdf = newdf.withColumn('Subproject',trim(newdf.Subproject))
newdf = newdf.withColumn('Subproject',split('Subproject','[/,]'))
newdf = newdf.withColumn('Subproject',explode('Subproject'))
newdf = newdf.withColumn('Subproject',substring_index('Subproject',' (',1))
#display(newdf)

StatementMeta(, cfa91a5a-fb11-439d-904a-a633ee1fa92e, 8, Finished, Available, Finished)

In [7]:
cleandf = newdf.select(
    col('ECR#'),\
    col('Urgency_Code_Mech'),\
    col('Products_Added'),\
    col('Drawing_Mech'),\
    col('Stage_Mech'),\
    col('Created'),\
    col('Modified'),\
    col('Project_Leader_Sign_Mech'),\
    col('ECR_Completed_Sign_Mech'),\
    col('Review_Project_Leader_Sign_Mech'),\
    col('Docu_Control_Leader_Sign_Mech'),\
    col('Docu_Control_Admin_Sign_Mech'),\
    col('NCR#'),\
    col('Total#ofFolders'),\
    col('Reason_For_ECR'),\
    col('Project'),\
    col('Subproject')
)

StatementMeta(, cfa91a5a-fb11-439d-904a-a633ee1fa92e, 9, Finished, Available, Finished)

In [8]:
delta_table_path = "Tables/ECR_List_Clean" #fill in your delta table path 
cleandf.write.format("delta").mode("overwrite").option('overwriteSchema','true').save(delta_table_path)

StatementMeta(, cfa91a5a-fb11-439d-904a-a633ee1fa92e, 10, Finished, Available, Finished)

In [9]:
delta_table_path = "Tables/ECR_List_Folders" #fill in your delta table path 
folderdf.write.format("delta").mode("overwrite").option('overwriteSchema','true').save(delta_table_path)

StatementMeta(, cfa91a5a-fb11-439d-904a-a633ee1fa92e, 11, Finished, Available, Finished)